In [1]:
import psutil
import numpy as np
from data.rosette_temp import Rosette
import pandas as pd
from multiprocess import Pool, get_context
from itertools import product
import os
import pyvista as pv

# Test #1: single rosette

In [19]:
a = 0.5
c = 1.0
r0 = 1.0
h0 = 0.25
hp = 0.25
n_arms = 5
ros = Rosette(a, c, r0, h0, hp, n_arms)
ros.unify_mesh()
pl = ros.plot()
pl.show()

Widget(value='<iframe src="http://localhost:62257/index.html?ui=P_0x30fc2c9b0_8&reconnect=auto" class="pyvista…

In [20]:
# calculate volume
v_ros = ros.volume
print(v_ros)

10.730433653341958


In [21]:
# calculate minimally bounding sphere
mbs = ros.calc_mbs()
print(mbs)

{'c': array([-2.95547695e-08, -5.55111512e-17, -5.77497894e-08]), 'r': np.float64(3.041381354288414), 'v': np.float64(117.84230984687372)}


In [22]:
# calculate effective density 
rho_eff = v_ros / mbs['v']
print(rho_eff)

0.09105756385194133


In [23]:
# visualize mbs
pl = ros.plot()
# create sphere with radius r using pyvista
sphere = pv.Sphere(radius=mbs['r'], center=mbs['c'])
# plot sphere with pyvista
pl.add_mesh(sphere, color='red', opacity=0.5)
pl.show()


Widget(value='<iframe src="http://localhost:62257/index.html?ui=P_0x336e7be30_9&reconnect=auto" class="pyvista…

# Test #2: test m

# Test #2: multiple rosettes, single cpu

# Test #3: multiple rosettes, multiple cpus